In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import math
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [2]:
def entropy(labels):
    counts = Counter(labels)
    total = len(labels)
    ent = 0.0
    for v in counts.values():
        p = v/total
        ent -= p * math.log2(p)
    return ent

In [3]:
def information_gain(X_col, y):
    base_entropy = entropy(y)
    values = set(X_col)
    total = len(y)
    weighted_entropy = 0
    for v in values:
        subset_y = [y[i] for i in range(total) if X_col[i] == v]
        weighted_entropy += (len(subset_y)/total) * entropy(subset_y)
    return base_entropy - weighted_entropy

In [4]:
def gain_ratio(X_col, y):
    ig = information_gain(X_col, y)
    values = set(X_col)
    total = len(y)
    split_info = -sum((list(X_col).count(v)/total) * math.log2(list(X_col).count(v)/total) for v in values)
    return ig / split_info if split_info != 0 else 0

In [5]:
def id3(X, y, attributes):
    if len(set(y)) == 1:
        return list(set(y))[0]  # pure class
    if not attributes:
        return Counter(y).most_common(1)[0][0]  # majority class
    
    # choose attribute with max info gain
    gains = [information_gain([row[attr] for row in X], y) for attr in attributes]
    best_attr = attributes[np.argmax(gains)]
    
    tree = {best_attr: {}}
    values = set(row[best_attr] for row in X)
    
    for v in values:
        sub_X = [row for i,row in enumerate(X) if row[best_attr] == v]
        sub_y = [y[i] for i,row in enumerate(X) if row[best_attr] == v]
        if not sub_X:
            tree[best_attr][v] = Counter(y).most_common(1)[0][0]
        else:
            new_attrs = [a for a in attributes if a != best_attr]
            tree[best_attr][v] = id3(sub_X, sub_y, new_attrs)
    return tree

# =====================
# C4.5 Algorithm
# =====================

def c45(X, y, attributes):
    if len(set(y)) == 1:
        return list(set(y))[0]
    if not attributes:
        return Counter(y).most_common(1)[0][0]
    
    # choose attribute with max gain ratio
    ratios = [gain_ratio([row[attr] for row in X], y) for attr in attributes]
    best_attr = attributes[np.argmax(ratios)]
    
    tree = {best_attr: {}}
    values = set(row[best_attr] for row in X)
    
    for v in values:
        sub_X = [row for i,row in enumerate(X) if row[best_attr] == v]
        sub_y = [y[i] for i,row in enumerate(X) if row[best_attr] == v]
        if not sub_X:
            tree[best_attr][v] = Counter(y).most_common(1)[0][0]
        else:
            new_attrs = [a for a in attributes if a != best_attr]
            tree[best_attr][v] = c45(sub_X, sub_y, new_attrs)
    return tree


In [6]:


def predict(tree, sample, default=None):
    """Predict the label for one sample (dict of features)."""
    if not isinstance(tree, dict):
        return tree  # leaf node

    # get the attribute at this node
    attr = next(iter(tree))
    branches = tree[attr]

    # if sample[attr] not in branches, return majority class of this node
    value = sample.get(attr)
    if value not in branches:
        # majority class fallback
        leaves = []
        def collect_labels(subtree):
            if isinstance(subtree, dict):
                for v in subtree.values():
                    collect_labels(v)
            else:
                leaves.append(subtree)
        collect_labels(branches)
        return Counter(leaves).most_common(1)[0][0]

    return predict(branches[value], sample, default)


In [7]:
df = pd.read_csv('petrol_consumption.csv')
display(df.head())

,Petrol_tax,Average_income,Paved_Highways,Population_Driver_licence(%),Petrol_Consumption
0,9.0,3571,1976,0.525,541
1,9.0,4092,1250,0.572,524
2,9.0,3865,1586,0.580,561
3,7.5,4870,2351,0.529,414
4,8.0,4399,431,0.544,410


In [9]:
features = df.drop(["Petrol_Consumption"], axis=1).columns.tolist()
X = df.drop(["Petrol_Consumption"], axis=1).to_dict("records")   # list of dicts
y = df["Petrol_Consumption"].tolist()

In [10]:
X

[{'Petrol_tax': 9.0,
  'Average_income': 3571,
  'Paved_Highways': 1976,
  'Population_Driver_licence(%)': 0.525},
 {'Petrol_tax': 9.0,
  'Average_income': 4092,
  'Paved_Highways': 1250,
  'Population_Driver_licence(%)': 0.572},
 {'Petrol_tax': 9.0,
  'Average_income': 3865,
  'Paved_Highways': 1586,
  'Population_Driver_licence(%)': 0.58},
 {'Petrol_tax': 7.5,
  'Average_income': 4870,
  'Paved_Highways': 2351,
  'Population_Driver_licence(%)': 0.529},
 {'Petrol_tax': 8.0,
  'Average_income': 4399,
  'Paved_Highways': 431,
  'Population_Driver_licence(%)': 0.544},
 {'Petrol_tax': 10.0,
  'Average_income': 5342,
  'Paved_Highways': 1333,
  'Population_Driver_licence(%)': 0.571},
 {'Petrol_tax': 8.0,
  'Average_income': 5319,
  'Paved_Highways': 11868,
  'Population_Driver_licence(%)': 0.451},
 {'Petrol_tax': 8.0,
  'Average_income': 5126,
  'Paved_Highways': 2138,
  'Population_Driver_licence(%)': 0.553},
 {'Petrol_tax': 8.0,
  'Average_income': 4447,
  'Paved_Highways': 8577,
  'Popu

In [11]:
y

[541,
 524,
 561,
 414,
 410,
 457,
 344,
 467,
 464,
 498,
 580,
 471,
 525,
 508,
 566,
 635,
 603,
 714,
 865,
 640,
 649,
 540,
 464,
 547,
 460,
 566,
 577,
 631,
 574,
 534,
 571,
 554,
 577,
 628,
 487,
 644,
 640,
 704,
 648,
 968,
 587,
 699,
 632,
 591,
 782,
 510,
 610,
 524]

In [12]:
# =====================
# Cross Validation
# =====================

def evaluate(X, y, algorithm_func):
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    metrics = {"accuracy": [], "f1": [], "precision": [], "recall": []}
    
    for train_idx, test_idx in kf.split(X):
        X_train, X_test = [X[i] for i in train_idx], [X[i] for i in test_idx]
        y_train, y_test = [y[i] for i in train_idx], [y[i] for i in test_idx]
        
        attributes = list(X[0].keys())
        tree = algorithm_func(X_train, y_train, attributes)
        
        y_pred = [predict(tree, sample) for sample in X_test]
        
        metrics["accuracy"].append((accuracy_score(y_test, y_pred)))
        metrics["f1"].append(f1_score(y_test, y_pred, average="macro", zero_division=0))
        metrics["precision"].append(precision_score(y_test, y_pred, average="macro", zero_division=0))
        metrics["recall"].append(recall_score(y_test, y_pred, average="macro", zero_division=0))
    
    return {k: round(np.mean(v),4) for k,v in metrics.items()}




In [13]:
id3_tree = id3(X,y, features)
print("=== ID3 Tree ===")
print(id3_tree)

=== ID3 Tree ===
{'Average_income': {3333: 554, 4870: 414, 5126: {'Petrol_tax': {8.0: 467, 7.5: 471}}, 3718: 714, 3721: 566, 3846: 631, 5002: 524, 3601: 534, 3865: 561, 3357: 628, 4512: 498, 4897: 464, 4258: 547, 3745: 591, 4391: 580, 4399: 410, 3063: 577, 3635: 648, 3640: 571, 3897: 704, 5319: 344, 3528: 487, 3656: 699, 4296: 610, 4300: 632, 4045: 640, 4817: 525, 3802: 644, 4188: 574, 4476: 510, 4318: 635, 5342: 457, 4447: 464, 4574: 460, 4449: 587, 5215: 782, 4332: 566, 4716: 865, 4206: 603, 4207: 508, 4593: 649, 3571: 541, 4341: 640, 4983: 540, 3448: 577, 4345: 968, 4092: 524}}


In [14]:
c45_tree = c45(X,y, features)
print("=== ID3 Tree ===")
print(c45_tree)

=== ID3 Tree ===
{'Average_income': {3333: 554, 4870: 414, 5126: {'Petrol_tax': {8.0: 467, 7.5: 471}}, 3718: 714, 3721: 566, 3846: 631, 5002: 524, 3601: 534, 3865: 561, 3357: 628, 4512: 498, 4897: 464, 4258: 547, 3745: 591, 4391: 580, 4399: 410, 3063: 577, 3635: 648, 3640: 571, 3897: 704, 5319: 344, 3528: 487, 3656: 699, 4296: 610, 4300: 632, 4045: 640, 4817: 525, 3802: 644, 4188: 574, 4476: 510, 4318: 635, 5342: 457, 4447: 464, 4574: 460, 4449: 587, 5215: 782, 4332: 566, 4716: 865, 4206: 603, 4207: 508, 4593: 649, 3571: 541, 4341: 640, 4983: 540, 3448: 577, 4345: 968, 4092: 524}}


In [15]:

print("ID3 CV Results:", evaluate(X, y, id3))
print("C4.5 CV Results:", evaluate(X, y, c45))
print("-------------------------")

ID3 CV Results: {'accuracy': np.float64(0.0), 'f1': np.float64(0.0), 'precision': np.float64(0.0), 'recall': np.float64(0.0)}
C4.5 CV Results: {'accuracy': np.float64(0.0), 'f1': np.float64(0.0), 'precision': np.float64(0.0), 'recall': np.float64(0.0)}
-------------------------


In [16]:
for x in X:
    print(predict(id3_tree,x))

metrics = {"accuracy": [], "f1": [], "precision": [], "recall": []}
        
y_pred = [predict(id3_tree, sample) for sample in X]
        
metrics["accuracy"].append((accuracy_score(y, y_pred)))
metrics["f1"].append(f1_score(y, y_pred, average="macro", zero_division=0))
metrics["precision"].append(precision_score(y, y_pred, average="macro", zero_division=0))
metrics["recall"].append(recall_score(y, y_pred, average="macro", zero_division=0))

print(metrics)

541
524
561
414
410
457
344
467
464
498
580
471
525
508
566
635
603
714
865
640
649
540
464
547
460
566
577
631
574
534
571
554
577
628
487
644
640
704
648
968
587
699
632
591
782
510
610
524
{'accuracy': [1.0], 'f1': [1.0], 'precision': [1.0], 'recall': [1.0]}


c:\Users\Admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:98: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  type_true = type_of_target(y_true, input_name="y_true")
c:\Users\Admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:99: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  type_pred = type_of_target(y_pred, input_name="y_pred")
c:\Users\Admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:98: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  type_true = type_of_target(y_true, input_name="y_true")
c:\Users\A

In [17]:
for x in X:
    print(predict(c45_tree,x))


metrics = {"accuracy": [], "f1": [], "precision": [], "recall": []}
        
y_pred = [predict(c45_tree, sample) for sample in X]
        
metrics["accuracy"].append((accuracy_score(y, y_pred)))
metrics["f1"].append(f1_score(y, y_pred, average="macro", zero_division=0))
metrics["precision"].append(precision_score(y, y_pred, average="macro", zero_division=0))
metrics["recall"].append(recall_score(y, y_pred, average="macro", zero_division=0))

print(metrics)

541
524
561
414
410
457
344
467
464
498
580
471
525
508
566
635
603
714
865
640
649
540
464
547
460
566
577
631
574
534
571
554
577
628
487
644
640
704
648
968
587
699
632
591
782
510
610
524
{'accuracy': [1.0], 'f1': [1.0], 'precision': [1.0], 'recall': [1.0]}


c:\Users\Admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:98: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  type_true = type_of_target(y_true, input_name="y_true")
c:\Users\Admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:99: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  type_pred = type_of_target(y_pred, input_name="y_pred")
c:\Users\Admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:98: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  type_true = type_of_target(y_true, input_name="y_true")
c:\Users\A